In [ ]:
# 1. Önce paketleri kur
!pip install  peft accelerate bitsandbytes transformers datasets

# 2. Sonra model fine-tuning kodunu çalıştır
from datasets import load_dataset
from transformers import AutoTokenizer

from peft import LoraConfig

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 39.3 MB/s eta 0:00:00


In [ ]:
!pip uninstall -U trl
!pip install -U trl==0.9.6


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -U
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 14.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-

In [ ]:
import trl
print(trl.__version__)

0.9.6


In [ ]:
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    AutoModelForCausalLM
)
from trl import SFTTrainer
from peft import LoraConfig
import torch

# Model ve Tokenizer
model_id = "TURKCELL/Turkcell-LLM-7b-v1"

# 4-bit quantization konfigürasyonu (A100 için BF16 ile)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Quantization ile modeli önceden yükle
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = 'right'  # SFTTrainer uyarısını gidermek için

# Dataset yükle
dataset = load_dataset("json", data_files="nihai_data.json", split="train")

# Formatting function
def formatting_func(example):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")

    if input_text:
        return f"Soru: {instruction}\nGirdi: {input_text}\nCevap: {output}"
    else:
        return f"Soru: {instruction}\nCevap: {output}"

dataset = dataset.map(lambda x: {"text": formatting_func(x)})

# LoRA config
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

# Optimize edilmiş Training args
training_args = TrainingArguments(
    output_dir="./outputs",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    report_to="none",
    max_grad_norm=0.3,
    warmup_ratio=0.03
)

# Trainer
trainer = SFTTrainer(
    model=model,  # Önceden yüklenmiş quantize edilmiş model
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=512,
    args=training_args
)

# CUDA bellek ayarları
torch.cuda.empty_cache()
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Eğitimi başlat
trainer.train()

# Modeli kaydet
trainer.save_model("./outputs")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/3331 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/trl/trainer/sft_trainer.py:413: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,3.310700
20,3.089100
30,3.053200
40,3.071400
50,2.729900
60,2.493700
70,2.247500
80,2.192000
90,2.058200
100,2.122400


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/

In [ ]:
from google.colab import drive

# Google Drive'ı /content/drive dizinine bağlar
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/outputs /content/drive/MyDrive/outputsdposuz

In [ ]:
!pip install --upgrade transformers==4.46.2
!pip install --upgrade trl==0.11.4
!pip install --upgrade peft==0.11.1
!pip install --upgrade accelerate==0.33.0

In [ ]:
!pip install -U peft trl transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 107.8 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.

In [ ]:
# === DPO: SFT sonrası LoRA adapter'larıyla devam ===
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from trl import DPOConfig, DPOTrainer
import torch, os

# ---- Model yolları ----
model_id = "TURKCELL/Turkcell-LLM-7b-v1"
sft_adapters_dir = "/content/outputs"      # SFT sonrası adapter'lar
dpo_output_dir   = "/content/outputs_dpo"  # DPO çıktıları buraya

# ---- 4-bit quantization ----
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ---- Tokenizer ----
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

# ---- Politika modeli (trainable) ----
base_policy = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)
policy_model = PeftModel.from_pretrained(base_policy, sft_adapters_dir, is_trainable=True)

# ---- Referans modeli (aynı SFT, donmuş) ----
base_ref = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quant_config,
    device_map="auto"
)
ref_model    = PeftModel.from_pretrained(base_ref, sft_adapters_dir, is_trainable=False)
for p in ref_model.parameters():
    p.requires_grad_(False)

# ---- DPO veri seti ----
# Beklenen kolonlar: "prompt", "chosen", "rejected"
dpo_dataset = load_dataset("json", data_files="dpo.json", split="train")
# Eğer farklı isimler varsa rename_columns kullan:
# dpo_dataset = dpo_dataset.rename_columns({"soru":"prompt","kazanan":"chosen","kaybeden":"rejected"})

# ---- DPO konfigürasyonu ----
dpo_args = DPOConfig(
    output_dir=dpo_output_dir,
    beta=0.1,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    logging_steps=10,
    save_strategy="epoch",
    bf16=True,
    fp16=False,
    max_length=1024,
    max_prompt_length=512,
    report_to=[]
)

# ---- CUDA ayarı ----
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# ---- DPO Trainer (eski & yeni TRL uyumlu) ----
try:
    # Yeni TRL (>=0.11) → processing_class parametresi
    dpo_trainer = DPOTrainer(
        model=policy_model,
        ref_model=ref_model,
        train_dataset=dpo_dataset,
        args=dpo_args,
        processing_class=tokenizer
    )
except TypeError:
    # Eski TRL (<=0.10.x) → tokenizer parametresi
    dpo_trainer = DPOTrainer(
        model=policy_model,
        ref_model=ref_model,
        train_dataset=dpo_dataset,
        args=dpo_args,
        tokenizer=tokenizer
    )

# ---- Eğitim ve kayıt ----
dpo_trainer.train()
dpo_trainer.save_model(dpo_output_dir)     # LoRA adapter'ları kaydedilir
tokenizer.save_pretrained(dpo_output_dir)  # tokenizer da aynı klasöre
dpo_trainer.save_state()                   # (opsiyonel) trainer state

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Step,Training Loss
10,0.121200
20,0.000100
30,0.000100
40,0.000400
50,0.000000
60,0.000000
70,0.000000
80,0.000000
90,0.000000
100,0.000000


In [ ]:
!cp -r /content/outputs_dpo /content/drive/MyDrive/outputs_dpo

In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 73.2 MB/s eta 0:00:00


In [ ]:
dataset = load_dataset("json", data_files="video.json", split="train")

In [ ]:
# Bu hücrede RAG için FAISS indekslerini kuruyoruz
# İki ayrı indeks: factual ve procedural
# Embedding modeli: BGE-M3 (çok dilli)

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embed_model = SentenceTransformer("BAAI/bge-m3")

# Dataseti güvenli şekilde böl (metadata kontrolü ile)
factual_data = [
    ex for ex in dataset
    if ex and isinstance(ex, dict)
    and "metadata" in ex
    and isinstance(ex["metadata"], dict)
    and ex["metadata"].get("task_type") == "factual"
]

procedural_data = [
    ex for ex in dataset
    if ex and isinstance(ex, dict)
    and "metadata" in ex
    and isinstance(ex["metadata"], dict)
    and ex["metadata"].get("task_type") == "procedural"
]

print(f"Factual kayıt sayısı: {len(factual_data)}")
print(f"Procedural kayıt sayısı: {len(procedural_data)}")

# Embedding ve FAISS index builder
def build_index(data):
    texts = [ex["output"] for ex in data if "output" in ex]
    embeddings = embed_model.encode(texts, normalize_embeddings=True)
    index = faiss.IndexFlatIP(embeddings.shape[1])   # cosine similarity için normalize_embeddings=True + inner product
    index.add(np.array(embeddings))
    return index, texts

factual_index, factual_texts = build_index(factual_data)
procedural_index, procedural_texts = build_index(procedural_data)

print("RAG indexleri hazır! ✅")

Factual kayıt sayısı: 755
Procedural kayıt sayısı: 13
RAG indexleri hazır! ✅


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from trl import DPOConfig, DPOTrainer
import torch, os

In [ ]:
model_id = "TURKCELL/Turkcell-LLM-7b-v1"

In [ ]:
# ---- Tokenizer ----
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.padding_side = "right"
if tokenizer.pad_token is None and tokenizer.eos_token is not None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# Bu hücrede LoRA'yı merge ediyoruz
from peft import AutoPeftModelForCausalLM
import torch

ft_model = AutoPeftModelForCausalLM.from_pretrained("./outputs", device_map="auto", torch_dtype=torch.float16)
ft_model = ft_model.merge_and_unload()

ft_model.save_pretrained("./merged_modeldposuz")
tokenizer.save_pretrained("./merged_modeldposuz")
print("LoRA merge edildi, HuggingFace formatında kaydedildi.")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

LoRA merge edildi, HuggingFace formatında kaydedildi.


In [ ]:
!cp -r /content/merged_modeldposuz /content/drive/MyDrive/merged_modeldposuz

In [ ]:
# GGUF dönüşümü (llama.cpp araçlarını kullanıyoruz)
# İlk olarak llama.cpp reposunu klonla
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# HF -> GGUF dönüşümü
!python /content/llama.cpp/convert_hf_to_gguf.py ./merged_model --outfile turkcell-erp.gguf

# Quantize (Q4_K_M)
!./quantize turkcell-erp.gguf turkcell-erp-q4_k_m.gguf Q4_K_M


Cloning into 'llama.cpp'...
remote: Enumerating objects: 59792, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 59792 (delta 48), reused 30 (delta 30), pack-reused 59716 (from 4)
Receiving objects: 100% (59792/59792), 157.20 MiB | 39.81 MiB/s, done.
Resolving deltas: 100% (42997/42997), done.
/content/llama.cpp/llama.cpp
Traceback (most recent call last):
  File "/content/llama.cpp/convert_hf_to_gguf.py", line 32, in <module>
    from mistral_common.tokens.tokenizers.base import TokenizerVersion
ModuleNotFoundError: No module named 'mistral_common'
/bin/bash: line 1: ./quantize: No such file or directory


In [ ]:
# Bu hücrede küçük bir test yapıyoruz
from transformers import AutoModelForCausalLM, pipeline

pipe = pipeline("text-generation", model="./merged_model", tokenizer="./merged_model", device=0)

system_prompt = """Kurallar:
- Eğer router procedural derse: 'UI Yolu' satırı + 1-2-3 adım, buton/metin adlarını arayüzde geçtiği gibi yaz.
- Eğer factual derse: Öz ve net anlat, en fazla 5 cümle.
- Ürün dışı sorularda kibarca sınır bildir.
"""

question = "Alınan çek için tahsilat makbuzu nasıl oluşturulur?"

prompt = f"### System:\n{system_prompt}\n\n### User:\n{question}\n\n### Assistant:\n"

out = pipe(prompt, max_new_tokens=300, do_sample=True, temperature=0.2)
print(out[0]["generated_text"])


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: './merged_model'.